In [ ]:
pip install pandas openpyxl pillow numpy

In [1]:
import pandas as pd
import openpyxl
from PIL import Image
from PIL import Image, ImageDraw, ImageChops, ImageFont
import os

In [3]:
category = "2025"

df = pd.read_excel("data/Partial Format Frame Auto.xlsx", sheet_name=category)
df.head()

# Update file only when there is new person, ask bday and take note of anniv

,full_name,short_name,k-id,birthday,anniversary,department,advance,pro,expert,habit_1,...,habit_4,habit_5,habit_6,shabit_5,th_2,habit_7,habit_8,habit_9,shabit_6,th_3
0,Mathew Sullivan,Mat Sullivan,m.sullivan,May,January,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Carter Guerrero,Cart Guerrero,c.guerrero,May,January,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Marley Dale,Marl Dale,m.dale,May,Febuary,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Harold Griffith,Harry Griffith,h.griffith,May,Febuary,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alysha Davidson,Aly Davidson,a.davidson,May,Febuary,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Values Birthday

In [5]:
# change value of this variable
bday_month = "May"
category = "birthday"

bday = df[(df[category] == bday_month)]

cleaned_bday = bday[["short_name", "k-id"]]
cleaned_bday

,short_name,k-id
0,Mat Sullivan,m.sullivan
1,Cart Guerrero,c.guerrero
2,Marl Dale,m.dale
3,Harry Griffith,h.griffith
4,Aly Davidson,a.davidson
5,Gerald Quinn,g.quinn
6,Mir Hernandez,m.hernandez
7,Mett Melendez,m.melendez
8,Sors Suarez,s.suarez
9,Bil Wlcox,b.wlcox


In [6]:
test = cleaned_bday.to_dict(orient='split', index=False)
bday = test['data']
bday

[['Mat Sullivan', 'm.sullivan'],
 ['Cart Guerrero', 'c.guerrero'],
 ['Marl Dale', 'm.dale'],
 ['Harry Griffith', 'h.griffith'],
 ['Aly Davidson', 'a.davidson'],
 ['Gerald Quinn', 'g.quinn'],
 ['Mir Hernandez', 'm.hernandez'],
 ['Mett Melendez', 'm.melendez'],
 ['Sors Suarez', 's.suarez'],
 ['Bil Wlcox', 'b.wlcox'],
 ['Lu Jefferson', 'l.jefferson'],
 ['Emman Hardy', 'e.hardy'],
 ['Jac Roborts', 'j.roborts']]

### Creating groups (to modify)

In [9]:
import pandas as pd

# Sample 2D array: [name, id]
data = bday
# Create DataFrame
df = pd.DataFrame(data, columns=["name", "id"])
groups = []

# Grouping logic
while not df.empty:
    n = len(df)
    for size in range(8, 0, -1):  # Try 8 → 1
        if n % size == 0 or n >= size:
            chunk = df.iloc[:size].reset_index(drop=True)
            groups.append(chunk)
            df = df.iloc[size:].reset_index(drop=True)
            break

# View results
for i, g in enumerate(groups, 1):
    print(f"\nGroup {i} (size {len(g)}):\n{g}")



Group 1 (size 8):
             name           id
0    Mat Sullivan   m.sullivan
1   Cart Guerrero   c.guerrero
2       Marl Dale       m.dale
3  Harry Griffith   h.griffith
4    Aly Davidson   a.davidson
5    Gerald Quinn      g.quinn
6   Mir Hernandez  m.hernandez
7   Mett Melendez   m.melendez

Group 2 (size 5):
           name           id
0   Sors Suarez     s.suarez
1     Bil Wlcox      b.wlcox
2  Lu Jefferson  l.jefferson
3   Emman Hardy      e.hardy
4   Jac Roborts    j.roborts


In [10]:
data = bday
groups = [] 
while data:
    n = len(data)
    for size in range(8, 0, -1):
        if n % size == 0 or n >= size:
            groups.append(data[:size])
            data = data[size:]
            break

for i, group in enumerate(groups, 1):
    print(f"Group {i} ({len(group)} members):", group)

Group 1 (8 members): [['Mat Sullivan', 'm.sullivan'], ['Cart Guerrero', 'c.guerrero'], ['Marl Dale', 'm.dale'], ['Harry Griffith', 'h.griffith'], ['Aly Davidson', 'a.davidson'], ['Gerald Quinn', 'g.quinn'], ['Mir Hernandez', 'm.hernandez'], ['Mett Melendez', 'm.melendez']]
Group 2 (5 members): [['Sors Suarez', 's.suarez'], ['Bil Wlcox', 'b.wlcox'], ['Lu Jefferson', 'l.jefferson'], ['Emman Hardy', 'e.hardy'], ['Jac Roborts', 'j.roborts']]


### Image Coordinates

In [17]:
single_y = 418
top_y = 328
bottom_y = 732

coord_arr_8 = [(51, top_y),(500, top_y),(948, top_y),(1396, top_y),
               (266, bottom_y),(714, bottom_y),(1163, bottom_y),(1163, bottom_y)]
coord_arr_7 = [(91, top_y),(589, top_y),(1087, top_y),(1585, top_y),
               (308, bottom_y),(838, bottom_y),(1367, bottom_y)]
coord_arr_6 = [(203, top_y),(733, top_y),(1262, top_y),
               (454, bottom_y),(983, bottom_y),(1513, bottom_y)]
coord_arr_5 = [(203, top_y), (838, top_y), (1472, top_y), 
               (518, bottom_y), (1158, bottom_y)]
coord_arr_4 = [(172, single_y),(616, single_y),(1060, single_y),(1504, single_y)]
coord_arr_3 = [(268, single_y),(838, single_y),(1408, single_y)]
coord_arr_2 = [(565, single_y),(1111, single_y)]
coord_arr_1 = [(838, single_y)]

### Generator

In [ ]:
def make_circular(image_path):
    img = Image.open(image_path).convert("RGBA")
    width, height = img.size
    
    mask = Image.new('L', (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0, width, height), fill=255)
    
    img.putalpha(mask)
    return img

# Load background
canvas = Image.open("backgrounds/bg1.png").convert("RGBA")

# Process person image
circular_image = make_circular("images/person-holder.jpg")
resize_image = 245
resized_image = circular_image.resize((resize_image, resize_image), Image.LANCZOS)


# Main process of 
counter = 0
while counter < len(coord_arr_5):   
    x = coord_arr_5[counter][0]
    y = coord_arr_5[counter][1]
    canvas.paste(resized_image, (x , y), resized_image)


    draw = ImageDraw.Draw(canvas)

    # Font 
    font = ImageFont.truetype("DejaVuSansMono.ttf", size=25)

    # Specify the text and its position
    text = feb_names[counter]
    y_text_offset = 300
    x_text_offset = 120
    text_position = (x + x_text_offset, y + y_text_offset)

    # Choose text color
    text_color = (0, 0, 0)

    draw.text(text_position, text, fill=text_color, font=font, anchor="mm")

    counter = counter + 1

# # canvas.save("[{department}]-{category}-{counter}", "JPEG")
# output_name = f"{department}-{category}-{counter}.jpeg"

# # Convert to RGB to save as JPEG
# canvas = canvas.convert("RGB")

# # Save to folder
# os.makedirs("output", exist_ok=True)
# canvas.save(os.path.join("output", output_name))

TODO: 
- calculate image and text offset to make the anchor point be the middle
- read the name and image path from the excel file 
- use the appropriate canvas when of number of people 

TODO: 
- Make this into a video, save it and get the duration 
- Set this duration in the software upload and observe how it will play

TODO:
- create folder for image and file in the downloads folder for easy access 
- detect if the file is an image or excel file 
- move them in their respective folders inside the project 